<a href="https://colab.research.google.com/github/Ahmedabdelwaly/NlpProject/blob/main/NlpHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets arabert bert_score evaluate rouge_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from arabert.preprocess import ArabertPreprocessor
from datasets import load_dataset
import torch

In [3]:
# Load 1% of OSCAR Arabic dataset
dataset = load_dataset(
    "oscar-corpus/oscar",
    "unshuffled_deduplicated_ar",
    split="train[:10000]"
)
# Train/Test split 90/10
dataset = dataset.train_test_split(test_size=0.1)

# Check dataset size
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/303k [00:00<?, ?B/s]

oscar.py:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

The repository for oscar-corpus/oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar-corpus/oscar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/9006977 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['id', 'text'],
        num_rows: 1000
    })
})


In [4]:
model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

In [5]:
# Load tokenizer & model
model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)

# Arabert Preprocessor
arabert_prep = ArabertPreprocessor(model_name=model_name)

# Preprocess texts
def preprocess_function(example):
    text = arabert_prep.preprocess(example["text"])
    return {"text": text}

dataset = dataset.map(preprocess_function)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
# Save train and test to text files (for TextDataset)
train_texts = "\n".join(dataset['train']['text'])
test_texts = "\n".join(dataset['test']['text'])

with open('train.txt', 'w', encoding='utf-8') as f:
    f.write(train_texts)

with open('test.txt', 'w', encoding='utf-8') as f:
    f.write(test_texts)


In [7]:
from transformers import TextDataset

# Load datasets
train_dataset = TextDataset(
    tokenizer=tokenizer, file_path='train.txt', block_size=64
)
test_dataset = TextDataset(
    tokenizer=tokenizer, file_path='test.txt', block_size=64
)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    save_steps=500,
    logging_steps=10,
    max_steps=500,  # <- only run 500 steps total
    report_to="none",
    fp16=True
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,9.235600
20,7.191000
30,6.541000
40,6.451300
50,6.155400
60,6.043900
70,5.932600
80,5.887000
90,5.822700
100,5.790300


TrainOutput(global_step=500, training_loss=5.731803634643555, metrics={'train_runtime': 85.8575, 'train_samples_per_second': 93.178, 'train_steps_per_second': 5.824, 'total_flos': 261292032000000.0, 'train_loss': 5.731803634643555, 'epoch': 0.10315659170621003})

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
import re

def predict(text, max_new_tokens=9, top_k=50, top_p=0.95, max_words=6):
    processed = arabert_prep.preprocess(text)
    input_ids = tokenizer.encode(processed, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Cleaning
    cleaned = re.sub(r'\b\d+\b', '', decoded)

    # 6 words max
    trimmed = " ".join(cleaned.split()[:max_words])
    return trimmed.strip()


In [10]:
output = predict("قررت المحكمة")
print(output)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


قررت المحكمة العسكرية في لاهاي الى


In [11]:
# Original sentence
output1 = predict("قررت المحكمة")
print(" قررت المحكمة →", output1)

# New test sentence 1
output2 = predict("أعلنت الحكومة")
print(" أعلنت الحكومة →", output2)

# New test sentence 2
output3 = predict("وصل الوفد المصري")
print(" وصل الوفد المصري →", output3)


 قررت المحكمة → قررت المحكمة الابتدائية في الرياض تقدم
 أعلنت الحكومة → أعلنت الحكومة اليوم الأحد ، عن
 وصل الوفد المصري → وصل الوفد المصري ، الذي وصل


In [12]:
import evaluate

# Load evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def evaluate_model(test_data):
    predictions = []
    references = []

    for sample in test_data:
        pred = predict(sample["input"], max_new_tokens=9)  # No [0] here!
        predictions.append(pred.strip())
        references.append([sample["reference"].strip()])

    bleu_score = bleu.compute(predictions=predictions, references=references)
    rouge_score = rouge.compute(predictions=predictions, references=[r[0] for r in references])

    print("BLEU Score:", bleu_score)
    print("ROUGE Score:", rouge_score)

    return {
        "BLEU": bleu_score,
        "ROUGE": rouge_score
    }

In [13]:
test_data = [
    {
        "input": "قررت المحكمة",
        "reference": "قررت المحكمة تأجيل الجلسة القادمة"
    },
    {
        "input": "أعلنت الحكومة",
        "reference": "أعلنت الحكومة بدء تنفيذ الخطة"
    },
    {
        "input": "وصل الوفد المصري",
        "reference": "وصل الوفد المصري صباح اليوم"
    }
]

# Evaluate model
results = evaluate_model(test_data)


BLEU Score: {'bleu': 0.0, 'precisions': [0.3888888888888889, 0.26666666666666666, 0.08333333333333333, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.2, 'translation_length': 18, 'reference_length': 15}
ROUGE Score: {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}


In [14]:
for i, sample in enumerate(test_data):
    print(f"\n Input: {sample['input']}")
    print(f" Reference: {sample['reference']}")
    print(f" Prediction: {predict(sample['input'], max_new_tokens=9)}")


 Input: قررت المحكمة
 Reference: قررت المحكمة تأجيل الجلسة القادمة
 Prediction: قررت المحكمة الشرعية من قبل محافظة

 Input: أعلنت الحكومة
 Reference: أعلنت الحكومة بدء تنفيذ الخطة
 Prediction: أعلنت الحكومة العراقية قبل ساعة عن

 Input: وصل الوفد المصري
 Reference: وصل الوفد المصري صباح اليوم
 Prediction: وصل الوفد المصري عن طريق العودة


In [35]:
from torch.nn import CrossEntropyLoss
import torch

def compute_perplexity(model, tokenizer, sentence):
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()


In [36]:
sentence = "قررت المحكمة تأجيل الجلسة القادمة"
ppl = compute_perplexity(model, tokenizer, sentence)
print(f" Perplexity: {ppl:.2f}")


 Perplexity: 51.08


Hugging face Gui

In [16]:
!pip install huggingface_hub


In [15]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

model_name = "aubmindlab/aragpt2-base"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

model.save_pretrained("./arabic_autocomplete_model")
tokenizer.save_pretrained("./arabic_autocomplete_model")


('./arabic_autocomplete_model/tokenizer_config.json',
 './arabic_autocomplete_model/special_tokens_map.json',
 './arabic_autocomplete_model/vocab.json',
 './arabic_autocomplete_model/merges.txt',
 './arabic_autocomplete_model/added_tokens.json',
 './arabic_autocomplete_model/tokenizer.json')

In [18]:
from huggingface_hub import notebook_login
notebook_login()


In [19]:
model.push_to_hub("AhmedAbdelwaly/arabic-autocomplete-model")
tokenizer.push_to_hub("AhmedAbdelwaly/arabic-autocomplete-model")


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AhmedAbdelwaly/arabic-autocomplete-model/commit/71af646fdce2638d036c5695b363a4355d743efc', commit_message='Upload tokenizer', commit_description='', oid='71af646fdce2638d036c5695b363a4355d743efc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AhmedAbdelwaly/arabic-autocomplete-model', endpoint='https://huggingface.co', repo_type='model', repo_id='AhmedAbdelwaly/arabic-autocomplete-model'), pr_revision=None, pr_num=None)

In [20]:
%%writefile app.py
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from arabert.preprocess import ArabertPreprocessor
import gradio as gr
import re

# Load fine-tuned model from Hugging Face Hub
model_name = "AhmedAbdelwaly/arabic-autocomplete-model"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

arabert_prep = ArabertPreprocessor(model_name=model_name)

def predict(text, max_new_tokens=9, max_words=6):
    processed = arabert_prep.preprocess(text)
    input_ids = tokenizer.encode(processed, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean and trim to 6 Arabic words
    no_numbers = re.sub(r'\b\d+\b', '', decoded)
    clean_text = re.sub(r'[^\w\s\u0600-\u06FF،؟]', '', no_numbers)
    trimmed = " ".join(clean_text.strip().split()[:max_words])
    return trimmed.strip()

demo = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=2, placeholder="اكتب بداية الجملة هنا..."),
    outputs="text",
    title="نموذج الإكمال التلقائي للغة العربية",
    description="أدخل بداية جملة وسيكملها النموذج باستخدام GPT-2 المعرب."
)

demo.launch()


Writing app.py


In [21]:
%%writefile requirements.txt
transformers
torch
arabert
gradio


Writing requirements.txt


In [37]:
# Save model and tokenizer to a directory
model.save_pretrained("./arabic_autocomplete_model")
tokenizer.save_pretrained("./arabic_autocomplete_model")


('./arabic_autocomplete_model/tokenizer_config.json',
 './arabic_autocomplete_model/special_tokens_map.json',
 './arabic_autocomplete_model/vocab.json',
 './arabic_autocomplete_model/merges.txt',
 './arabic_autocomplete_model/added_tokens.json',
 './arabic_autocomplete_model/tokenizer.json')

In [38]:
!zip -r arabic_autocomplete_model.zip arabic_autocomplete_model


  adding: arabic_autocomplete_model/ (stored 0%)
  adding: arabic_autocomplete_model/config.json (deflated 52%)
  adding: arabic_autocomplete_model/generation_config.json (deflated 23%)
  adding: arabic_autocomplete_model/model.safetensors (deflated 7%)
  adding: arabic_autocomplete_model/merges.txt (deflated 77%)
  adding: arabic_autocomplete_model/special_tokens_map.json (deflated 52%)
  adding: arabic_autocomplete_model/vocab.json (deflated 75%)
  adding: arabic_autocomplete_model/tokenizer.json (deflated 85%)
  adding: arabic_autocomplete_model/tokenizer_config.json (deflated 74%)


In [39]:
from google.colab import files
files.download("arabic_autocomplete_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>